In [1]:
import modin.pandas as pd
import nums
from nums import numpy as nps

from nums.experimental.nums_modin import from_modin

import warnings
warnings.filterwarnings("ignore");

nums.init()

2021-11-02 18:14:02,232	INFO services.py:1265 -- View the Ray dashboard at http://127.0.0.1:8265


Using driver node ip as head node.
head node 10.0.0.4
total cpus 30.0
device_grid (0, 0) 0=node:10.0.0.4/cpu:1


In [2]:
def rmse(actual, expected):
    import numpy as np
    if type(actual) == type(nps.array([])):
        return nps.sqrt(nps.mean((expected - actual) ** 2)).get()
    elif type(actual) == type(np.array([])):
        return np.sqrt(np.mean((expected - actual) ** 2))    

In [3]:
%%time
X_data = pd.DataFrame()
y_data = pd.DataFrame()

for year in range(2020, 2021):
    X_data = X_data.append(pd.read_csv("data/X_" + str(year) + ".csv.gz", compression='gzip'))
    y_data = y_data.append(pd.read_csv("data/y_bin_" + str(year) + ".csv.gz", compression='gzip'))

CPU times: user 3.01 s, sys: 707 ms, total: 3.72 s
Wall time: 4.77 s


In [4]:
X_data.shape

(3272224, 8)

In [5]:
y_data.shape

(3272224, 1)

In [6]:
X = from_modin(X_data).astype(float)
y = from_modin(y_data).astype(float).reshape(-1)

In [7]:
shuffle = nps.random.permutation(X.shape[0])
X = X[shuffle]
y = y[shuffle]
split = int(X.shape[0] * 0.8)
X_train = X[:split]
X_test = X[split:]
y_train = y[:split]
y_test = y[split:]

In [8]:
from nums.models.glms import LogisticRegression

model = LogisticRegression(solver="newton", penalty="l2", tol=1e-4)

1 years: Wall time: 5.41 s
2 years: Wall time: 9.8 s
3 years: Wall time: 18.8 s

In [9]:
%%time
model.fit(X_train, y_train)

CPU times: user 4.7 s, sys: 1.82 s, total: 6.52 s
Wall time: 5.41 s


In [10]:
%%time
training_results = model.predict(X_train)

CPU times: user 143 ms, sys: 27.8 ms, total: 170 ms
Wall time: 63.7 ms


In [11]:
%%time
test_results = model.predict(X_test)

CPU times: user 45.6 ms, sys: 1.47 ms, total: 47.1 ms
Wall time: 16.1 ms


In [12]:
print("Training accuracy: ", nps.sum(training_results == y_train).get() / y_train.shape[0])
print("Testing accuracy:  ", nps.sum(test_results == y_test).get() / y_test.shape[0])

Training accuracy:  0.7215792471404194
Testing accuracy:   0.7215900495839986


In [13]:
X_train_np = X_train.get()
X_test_np = X_test.get()
y_train_np = y_train.get()
y_test_np = y_test.get()

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
model = LogisticRegression()

1 years: 1min 46s
2 years: 3min 31s
3 years: 7min 4s

In [16]:
%%time
model.fit(X_train_np, y_train_np)

CPU times: user 8min 30s, sys: 8min 16s, total: 16min 46s
Wall time: 1min 46s


LogisticRegression()

In [17]:
%%time
training_results = model.predict(X_train_np)

CPU times: user 3.46 s, sys: 3.82 s, total: 7.28 s
Wall time: 168 ms


In [18]:
%%time
test_results = model.predict(X_test_np)

CPU times: user 1.26 s, sys: 1.61 s, total: 2.87 s
Wall time: 50.3 ms


In [23]:
print("Training accuracy: ", model.score(X_train_np, y_train_np))
print("Testing accuracy:  ", model.score(X_test_np, y_test_np))

Training accuracy:  0.701938337172856
Testing accuracy:   0.7017539990145873
